An example of PCH on an instance of $m_{1}= m_{2} = 2$; $j_{1} = 10$ and $j_{2} = 8$.

In [9]:
n1 = 10 #Number of jobs in first stage
n2 = 8  #Number of jobs in second stage
m1 = 2  #Number of machines in first stage
m2 = 2  #Number of machines in second stage

j1 = list(range(n1)) # First stage jobs
j2 = list(range(n2)) # Second stage jobs
p1 = [58,27,16,81,69,68,62,13,42,96] # Processing times of first stage jobs
p2 = [61,46,81,48,23,23,15,75] # Processing times of second stage jobs

#Matrix of precedents (rows) and sucessors (lines). eg. Sucessors of j1[0] are 1,2,4,5,6,7 and 8.
prec = [
[1,1,1,1,1,0,1,1,0,1],
[1,0,1,0,0,1,0,1,1,1],
[0,1,1,0,1,0,1,1,0,0],
[1,1,1,1,1,1,0,1,1,1],
[1,1,0,1,0,1,0,0,0,1],
[1,0,0,1,1,0,1,0,0,1],
[1,0,0,1,0,1,1,0,0,1],
[1,1,1,1,0,1,0,0,1,1]]

Step 1: Delta computation

In [10]:
delta = [0] * n1
for j in range(n1):
	delta_aux = 0
	for i in range(n2):
		if prec[i][j] == 1:
			delta[j] += p2[i]
			if p2[i] > delta_aux:
				delta_aux = p2[i]
	delta[j] = max(delta_aux, delta[j]/m2)
print("Deltaj =", delta)

Deltaj = [145.5, 144.0, 155.5, 122.5, 106.5, 103.5, 90.0, 118.0, 84.5, 145.5]


Step 2: Sort and compute cut limit for preemption

In [11]:
import math

sum_p = 0
cm = 0
max_1 = 0
max_2 = 0

#Sort according to LPT rule
j1 = [x for _,x in sorted(zip(p1, j1), reverse=True)]
#Sort according to shortest delta first
j1 = [x for _,x in sorted(zip(delta, j1))]
#j1 is sorted in shortest delta first and using LPT to break ties.

for j in j1:
	sum_p += p1[j]
	cm = max(sum_p/m1+delta[j], cm)
	if cm > max_1:
		max_1 = cm
	if p1[j] + delta[j] > max_2:
		max_2 = p1[j] + delta[j]

cut_limit = math.ceil(max(max_1, max_2))
cut_limit

377

Step 3: Sequence the first stage

In [12]:
import operator

machines = [[-1 for x in range(n1+1)] for y in range(m1)]
for i in range(m1): machines[i][0] = cut_limit

machines_p1 = [[-1 for x in range(n1+1)] for y in range(m1)]
for i in range(m1): machines_p1[i][0] = 0
    
for i in j1:
    p1_i = p1[i]
    delta_i = delta[i]
    cm = 0
    
    while True:
        idx_m, value = max(enumerate([machines[j][0] for j in range(m1)]), key=operator.itemgetter(1))
        idx_p = 1
        while machines[idx_m][idx_p] != -1:
            idx_p += 1
            
        machines[idx_m][idx_p] = i
        cm = machines_p1[idx_m][0] + p1_i + delta_i
        if cm <= cut_limit:
            machines_p1[idx_m][0] += p1_i;
            machines_p1[idx_m][idx_p] = p1_i;
            machines[idx_m][0] -= p1_i;
            p1_i = 0;
        else:
            machines_p1[idx_m][idx_p] = p1_i - (cm - cut_limit);
            machines_p1[idx_m][0] += machines_p1[idx_m][idx_p];
            machines[idx_m][0] -= machines_p1[idx_m][idx_p];
            p1_i = (cm - cut_limit);
        if p1_i <= 0: break
print("First stage:")
for m in machines:
    print(m)

First stage:
[106.5, 2, 0, 1, 5, 7, 9, 4, -1, -1, -1]
[115.5, 8, 6, 3, 9, 4, -1, -1, -1, -1, -1]


Step 4: Define preemptions